In [1]:
import cv2
import numpy as np
import os
import yaml
from yaml import SafeLoader

In [2]:
# Load yaml
with open('data.yaml',mode='r') as f:
    data_yaml = yaml.load(f,Loader=SafeLoader)
    
labels = data_yaml['names']
print(labels)


['BIODEGRADABLE', 'GLASS', 'PLASTIC', 'METAL', 'CARDBOARD', 'PAPER']


In [3]:
# Load YOLO model
yolo = cv2.dnn.readNetFromONNX('./Model2/weights/best.onnx')
yolo.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
yolo.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

In [4]:
# Load the image
img = cv2.imread('/Users/sujalnagpal/Desktop/Internet of Things 2/2_Predictions/Screenshot 2024-03-15 at 10.03.46 AM.png')
image = img.copy()
row, col, d = image.shape

# Get the YOLO prediction from the image
# step-1 convert image into square image (array)
max_rc = max(row, col)
input_image = np.zeros((max_rc, max_rc, 3), dtype=np.uint8)
input_image[0:row, 0:col, :] = image
# ste-2: get predictions from square array
INPUT_WH_YOLO = 640
blob = cv2.dnn.blobFromImage(input_image,1/255,(INPUT_WH_YOLO,INPUT_WH_YOLO),swapRB=True,crop=False)
yolo.setInput(blob)
preds = yolo.forward() # detection or prediction from YOLO

In [5]:
print(preds.shape)

(1, 25200, 11)


In [6]:
# Non maximum Supression
# step-1: filter detection based on confidence (0.4) and probability score (0.25)
detections = preds[0]
boxes = []
confidences = []
classes = []

# width and height of the image (input_image)
image_w, image_h = input_image.shape[:2]
x_factor = image_w/INPUT_WH_YOLO
y_factor = image_h/INPUT_WH_YOLO


for i in range(len(detections)):
    row = detections[i]
    confidence = row[4] # confidence of detection an object
    if confidence > 0.0:
        class_score = row[5:] #maximum probability from one object
        class_id = row[5:].argmax() # get the index position at which max probability occur
        
        if np.any(class_score > 0.25):

            cx,cy,w,h = row[0:4]
            # construct bounding box from four values
            # left, top, width and height
            left = int((cx - 0.5*w)*x_factor)
            top = int((cy - 0.5*h)*y_factor)
            width = int(w*x_factor)
            height = int(h*y_factor)
            
            box = np.array([left,top,width,height])
            
            # append values into the list
            confidences.append(confidence)
            boxes.append(box)
            classes.append(class_id)
            
# clean
boxes_np = np.array(boxes).tolist()
confidences_np = np.array(confidences).tolist()

#NMS
index = cv2.dnn.NMSBoxes(boxes_np,confidences_np,0.25,0.45).flatten()            


In [7]:
len(index)

16

In [8]:
# Draw the bounding boxes
for ind in index:
    print(ind)
    x, y, w, h = boxes_np[ind]
    bb_conf = confidences_np[ind]
    classes_id = classes[ind]
    class_name = labels[classes_id]
    
    text = f'{class_name}: {bb_conf}%'
    print(text)
    
    cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
    cv2.rectangle(image, (x, y-30), (x+w, y), (255, 255, 255), -1)
    
    cv2.putText(image, text, (x, y-10), cv2.FONT_HERSHEY_PLAIN, 0.7, (0, 0, 0), 1)


22120
GLASS: 0.9191854596138%
22103
PLASTIC: 0.8884083032608032%
22125
PLASTIC: 0.8827975392341614%
5743
METAL: 0.880513608455658%
22135
GLASS: 0.8782142996788025%
22154
GLASS: 0.8740674257278442%
22168
PLASTIC: 0.8635746836662292%
18856
PLASTIC: 0.8329452872276306%
6534
PLASTIC: 0.8319578766822815%
7172
METAL: 0.822797954082489%
17105
PLASTIC: 0.8125244379043579%
7125
METAL: 0.7817591428756714%
18873
METAL: 0.7787566781044006%
5875
METAL: 0.7361181974411011%
21747
PLASTIC: 0.7186959981918335%
22117
GLASS: 0.7077674865722656%


In [ ]:
cv2.imshow('original',img)
cv2.imshow('yolo_prediction',image)
cv2.waitKey(0)
cv2.destroyAllWindows()